In [1]:
import pandas as pd
data = pd.read_csv('thanksgiving.csv', encoding='Latin-1')

In [3]:
print(data.columns[:3])

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?'],
      dtype='object')


In [5]:
data['Do you celebrate Thanksgiving?'].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [12]:
yes_values = data['Do you celebrate Thanksgiving?'] == 'Yes'
data = data.loc[yes_values,:]
data['Do you celebrate Thanksgiving?'].value_counts()

Yes    980
Name: Do you celebrate Thanksgiving?, dtype: int64

In [13]:
data['What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [17]:
tofurkey_filter = data['What is typically the main dish at your Thanksgiving dinner?'] == 'Tofurkey'
tofurkey_responses = data.loc[tofurkey_filter]
tofurkey_responses['Do you typically have gravy?']

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

In [29]:
apple_is_null = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple'])
pumpkin_is_null = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin'])
pecan_is_null = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])

ate_pies = apple_is_null & pumpkin_is_null & pecan_is_null
ate_pies.value_counts()

False    876
True     104
dtype: int64

In [31]:
def dataCleaner(data):
    if pd.isnull(data):
        return None
    values = data.split(' ')
    if len(values) == 1:
        return int(values[0].replace('+', ''))
    else:
        return int(values[0])

data['int_age'] = data['Age'].apply(lambda x: dataCleaner(x))
data['int_age'].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

The results above do not tell the whole story. The data collected did not take the actual ages of the survey participants. The max value is not a true max since the encompasses 60+ participants.

In [34]:
def incomeDataCleaner(data):
    if pd.isnull(data):
        return None
    value = data.split(' ')[0]
    if value == 'Prefer':
        return None
    value = value.replace('$', '')
    value = value.replace(',', '')
    return int(value)

data['int_income'] = data['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(lambda x: incomeDataCleaner(x))
data['int_income'].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

Again the results above do not tell the whole story. Income was collected in ranges. Information is not verified. 

In [35]:
less_than_150k_filter = data['int_income'] < 150000
data.loc[less_than_150k_filter]['How far will you travel for Thanksgiving?'].value_counts()


Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [36]:
greater_than_150k_filter = data['int_income'] > 150000
data.loc[greater_than_150k_filter]['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

The data shows a significant difference between those that are will to travel who make less than 150k than those above that income level. I would be interesting to see how the ages correlate against that breakdown. This could be an indication that those that are earning above 150k are expected to be the hosts for Thanksgiving in their families. Or that family is not as valued as those who have earn/have less.
---------------
After deeper consideration. We would have to normalize the data for their respective categories. The difference may not be as big as the numbers initially appear to be.

In [46]:
data.pivot_table(index='Have you ever tried to meet up with hometown friends on Thanksgiving night?', columns='Have you ever attended a "Friendsgiving?"', values='int_age')

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [47]:
data.columns

Index(['RespondentID', 'Do you celebrate Thanksgiving?',
       'What is typically the main dish at your Thanksgiving dinner?',
       'What is typically the main dish at your Thanksgiving dinner? - Other (please specify)',
       'How is the main dish typically cooked?',
       'How is the main dish typically cooked? - Other (please specify)',
       'What kind of stuffing/dressing do you typically have?',
       'What kind of stuffing/dressing do you typically have? - Other (please specify)',
       'What type of cranberry saucedo you typically have?',
       'What type of cranberry saucedo you typically have? - Other (please specify)',
       'Do you typically have gravy?',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Brussel sprouts',
       'Which of these side dishes aretypically served at your Thanksgiving dinner? Please select all that apply. - Carrots',
       'Which of these side dishes aretypically served

In [49]:
data.pivot_table(index='Have you ever tried to meet up with hometown friends on Thanksgiving night?', columns='Have you ever attended a "Friendsgiving?"', values='int_income')

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842


Not completely sure how to interpret the numbers. The questions used the 'ever' which could mean many Thanksgivings ago. If users are more social at a younger ages than older this result would not support that claim. 

I do wonder if those that appear to be anti-social, (No-No) are also career driven